In [1]:
import sys
sys.path.append("../tools/")
import pdtb_utils
import pandas as pd
test_relations = pdtb_utils.get_train_relations()

In [2]:
df = pd.DataFrame([{'id': rel.relation_id(), 
                    'senses': tuple(rel.senses()), 
                    'relation_type': rel.relation_type(), 
                    'connective_token': rel.connective_token()} 
                   for rel in test_relations]).set_index('id')
df.describe()

,connective_token,relation_type,senses
count,32535,32535,32535
unique,754,4,88
top,,Explicit,"(Expansion.Conjunction,)"
freq,4133,14722,7355


In [3]:
df.groupby('relation_type').describe()

connective_token                    senses
relation_type                                                  
AltLex        count               524                       524
              unique              415                        18
              top      That's because      (Contingency.Cause,)
              freq                 11                       220
EntRel        count              4133                      4133
              unique                1                         1
              top                                     (EntRel,)
              freq               4133                      4133
Explicit      count             14722                     14722
              unique              324                        67
              top                 and  (Expansion.Conjunction,)
              freq               1811                      4126
Implicit      count             13156                     13156
              unique               95                        54
              top             because      (Contingency.Cause,)
              freq               1537                      3387

In [26]:
freq = pd.DataFrame(df.groupby('senses').count().sort_values('connective_token', ascending=False)['connective_token'])
freq['ratio'] = freq['connective_token'] / freq['connective_token'].sum()
with open('../paper/tables/sense_frequency.tex', 'w') as w:
    freq.head(20).to_latex(w)

In [27]:
pd.DataFrame.to_latex?

## Initial thoughts

- Temporal contains typical temporal words, "years", "25 days later", etc. This should be fairly easy to solve by asking if it contains a certain set of temporal words.